Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

train = pd.read_csv('https://raw.githubusercontent.com/BrokenShell/DS-Unit-4-Sprint-1-NLP/master/module3-document-classification/data/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/BrokenShell/DS-Unit-4-Sprint-1-NLP/master/module3-document-classification/data/test.csv')

In [2]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


### Define Pipeline Components

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [4]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [5]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=8, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    4.9s
[Parallel(n_jobs=8)]: Done 160 out of 160 | elapsed:   18.9s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [6]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [7]:
grid_search.best_score_

0.7191128016208144

In [8]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [9]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [10]:
subNumber = 3

In [11]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

In [12]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc']
data = fetch_20newsgroups(subset='train', categories=categories)

In [13]:
svd = TruncatedSVD(n_components=100, algorithm='randomized', n_iter=10)

params = { 
    'lsi__svd__n_components': [10, 100, 250],
    'lsi__vect__max_df': [0.9, 0.95, 1.0],
    'clf__n_estimators': [5, 10, 20],
}

lsi = Pipeline([('vect', vect), ('svd', svd)])
pipe = Pipeline([('lsi', lsi), ('clf', clf)])

grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=8, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  2.3min


KeyboardInterrupt: 

In [ ]:
grid_search.best_score_

In [ ]:
# submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
# submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
# subNumber += 1

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

### Define Pipeline Components

In [14]:
svd = TruncatedSVD(n_components=100, algorithm='randomized', n_iter=10)
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

lsi = Pipeline([('vect', vect), ('svd', svd)])

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [15]:
parameters = {
    'lsi__svd__n_components': [10, 100, 250],
    'lsi__vect__max_df': (0.75, 1.0),
    'clf__max_depth': (5, 10, 15, 20),
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=8, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  2.3min
[Parallel(n_jobs=8)]: Done 120 out of 120 | elapsed:  9.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

### Make a Submission File

In [16]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [17]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [18]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [19]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

In [21]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [22]:
# if this cell wont run - restart the runtime!
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import spacy

nlp = spacy.load("en_core_web_lg")

In [23]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [24]:
!pip install IteratorAlgorithms

In [25]:
import IteratorAlgorithms as ia

In [26]:
param_dist = {
    'max_depth' : [4, 8, 16, 32, None],
    'min_samples_leaf' : [1, 2, 4, 6],
    'max_features': [6, 9, 12, 'auto']
}
iterations = ia.product(len(n) for n in param_dist.values())
print(iterations)

80


In [27]:
clf = RandomForestClassifier(random_state=42, n_jobs=8)
X = get_word_vectors(train['description'])

search = RandomizedSearchCV(
    clf, 
    param_dist, 
    cv=5, 
    n_jobs=-1, 
    verbose=10, 
    random_state=42, 
    n_iter=iterations,
).fit(X, train['ratingCategory'])

Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   22.2s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   26.9s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   31.3s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   35.7s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   41.3s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

In [28]:
print(search.best_params_)
print(search.best_score_)

{'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 16}
0.7355028983729011


### Make a Submission File

In [29]:
# Predictions on test sample
pred = search.predict(get_word_vectors(test['description']))

In [30]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [31]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [32]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?